In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import math
import re
import string
import numpy as np
import pandas as pd
import random
import json
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, MultiStepLR, CosineAnnealingWarmRestarts
import nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate import meteor

from data import *
from encoder_decoder import *
from train import *
from eval import *
from utils import *
from neuro_dec import *

# required for bleu
# nltk.download("wordnet")

/home/junn/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SEED = 31989101
HIDDEN_SIZE = 256
MAX_INGR_LEN = 150 # fixed from assignment
MAX_RECIPE_LEN = 600
DROPOUT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## ensuring reproducibility
def reset_rng():
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

reset_rng()

# to easily read ingredients and instructions
pd.set_option('display.max_colwidth', 2000)

print(f"Using device: {DEVICE}")

Using device: cuda


In [4]:
data_root = "./Cooking_Dataset"
add_intermediate_tag=False

train_df_orig = pd.read_csv(os.path.join(data_root, "train.csv"), usecols=['Ingredients', 'Recipe'])
dev_df_orig = pd.read_csv(os.path.join(data_root, "dev.csv"), usecols=['Ingredients', 'Recipe'])
test_df_orig = pd.read_csv(os.path.join(data_root, "test.csv"), usecols=['Ingredients', 'Recipe'])
train_df = preprocess_data(train_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)
dev_df = preprocess_data(dev_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)
test_df = preprocess_data(test_df_orig, max_ingr_len=MAX_INGR_LEN, max_recipe_len=MAX_RECIPE_LEN, add_intermediate_tag=add_intermediate_tag)
vocab = Vocabulary(add_intermediate_tag=add_intermediate_tag)
vocab.populate(train_df)
vocab.n_unique_words
train_ds = RecipeDataset(train_df, vocab)
# subset_train_ds = RecipeDataset(train_df[:250], vocab) # ! REMOVE LATER
dev_ds_val_loss = RecipeDataset(dev_df, vocab, train=True) # used for getting validation loss
dev_ds_val_met = RecipeDataset(dev_df, vocab, train=False) # used for getting validation BLEU, and other metrics
test_ds = RecipeDataset(test_df, vocab, train=False)

Number of data samples before preprocessing: 101340
Number of data samples after preprocessing: 100637 (99.306%)
Number of data samples before preprocessing: 797
Number of data samples after preprocessing: 793 (99.498%)
Number of data samples before preprocessing: 778
Number of data samples after preprocessing: 774 (99.486%)


100%|██████████| 100637/100637 [00:03<00:00, 32516.58it/s]


In [25]:
embedding_size=300
num_layers=3
encoder_multilayer_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, 
                          padding_value=vocab.word2index(PAD_WORD), num_lstm_layers=num_layers).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# also it should never generate "<UNKNOWN>"
# decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
decoder_multilayer_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, 
                              padding_val=vocab.word2index(PAD_WORD), dropout=DROPOUT, num_lstm_layers=num_layers).to(DEVICE)

In [26]:
load_model(encoder_multilayer_attn, decoder_multilayer_attn, "multilayer_attn_adam_without_intermediate_tags_wd0_lr1e-3_ep_14")

In [5]:
# embedding_size=300
# encoder_attn = EncoderRNN(vocab.n_unique_words, embedding_size=embedding_size, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index(PAD_WORD)).to(DEVICE)
# # in the training script, decoder is always fed a non-end token and thus never needs to generate padding
# # also it should never generate "<UNKNOWN>"
# # decoder = DecoderRNN(embedding_size=embedding_size,hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-2).to(DEVICE)
# decoder_attn = AttnDecoderRNN(embedding_size, hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1, padding_val=vocab.word2index(PAD_WORD), 
#                               dropout=DROPOUT).to(DEVICE)

In [27]:
all_ingredients_list = get_all_ingredients("./ingredient_set.json")

In [19]:
all_decoder_outs_norm, all_gt_recipes_norm = eval(encoder_multilayer_attn, decoder_multilayer_attn, test_ds, vocab, batch_size=128, decoder_mode="attention",
                                        max_recipe_len=MAX_RECIPE_LEN)

100%|██████████| 7/7 [00:02<00:00,  2.59it/s]


In [23]:
calc_bleu(all_decoder_outs_norm, all_gt_recipes_norm)

0.053654397203939004

In [29]:
alpha=50 # likelihood
neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty = 10, 0.5, 0.5
best_bleu = 0
best_bleu_params = None
for k in [3, 4, 5]:
    for beta in [2, 4]:
        for lam in [1, 3, 5, 7]:
            print(f"k={k}, beta={beta}, lam={lam}")
            all_decoder_outs, all_gt_recipes, all_gt_ings = eval_neuro_decoding(
                encoder_multilayer_attn, decoder_multilayer_attn, test_ds, vocab, all_ingredients_list, k=k, alpha=alpha, beta=beta, 
                neg_constraint_penalty=neg_constraint_penalty, likelihood_penalty=likelihood_penalty, 
                low_irr_satisfaction_penalty=low_irr_satisfaction_penalty, lam=lam)
            bleu = calc_bleu(all_gt_recipes,all_decoder_outs)
            print(bleu)
            if bleu > best_bleu:
                best_bleu = bleu
                best_bleu_params = dict(k=k, beta=beta, lam=lam)

k=3, beta=2, lam=1


100%|██████████| 774/774 [15:48<00:00,  1.23s/it]


0.056226539437320054
k=3, beta=2, lam=3


 89%|████████▉ | 688/774 [13:40<01:42,  1.19s/it]


KeyboardInterrupt: 

In [17]:
k=3
alpha=50 # likelihood
beta = 4 # num constraints satisfied
neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty = 10, 0.5, 0.5
lam = 5.0

all_decoder_outs, all_gt_recipes, all_gt_ings = eval_neuro_decoding(
    encoder_multilayer_attn, decoder_multilayer_attn, test_ds, vocab, all_ingredients_list, k=k, alpha=alpha, beta=beta, 
    neg_constraint_penalty=neg_constraint_penalty, likelihood_penalty=likelihood_penalty, 
    low_irr_satisfaction_penalty=low_irr_satisfaction_penalty, lam=lam)

# with torch.no_grad():
#     final_decoder_out_txt = eval_neuro_decoding_iter(ingredients, ing_lens, encoder_attn, decoder_attn,
#                             vocab, pos_constraints, neg_constraints, max_recipe_len=MAX_RECIPE_LEN,
#                             k=k, alpha=alpha, beta=beta, neg_constraint_penalty=neg_constraint_penalty,
#                             likelihood_penalty=likelihood_penalty, low_irr_satisfaction_penalty=low_irr_satisfaction_penalty,
#                             lam=lam)

100%|██████████| 774/774 [16:19<00:00,  1.27s/it]


In [40]:
all_gt_ings[8]

['<INGREDIENT_START> 20 ears tender corn ; - = or = - 2 1/2 qt - frozen whole kernel corn , thawed 1 1/4 c chopped onions 1 c chopped green bell peppers 1 c chopped red bell peppers 1 c chopped celery 2 2/3 c white vinegar 2 c water 1 1/2 c sugar 4 1/2 ts mustard seeds 1 tb salt 1 ts celery seeds 1/2 ts ground turmeric <INGREDIENT_END>']

In [11]:
all_decoder_outs[10]

['<RECIPE_START>',
 'combine',
 'apples',
 ',',
 'cinnamon',
 ',',
 'sugar',
 ',',
 'cornstarch',
 'and',
 'cinnamon',
 'in',
 'a',
 'large',
 'bowl',
 'pour',
 'over',
 'apples',
 'bake',
 'at',
 '350',
 'for',
 '30',
 'minutes',
 'or',
 'until',
 'apples',
 'are',
 'tender',
 '<RECIPE_END>']

In [18]:
calc_bleu(all_gt_recipes, all_decoder_outs)

0.0552057208131182

---

In [8]:
constraints_dict = {}

num_invalid_ingredients = 0
for ingredient in all_ingredients_list:
    invalid_ingredient=False
    ingredient_idx_form = []

    for word in ingredient.split():
        if not vocab.word_exist_in_vocab(word):
            invalid_ingredient = True
            break
        ingredient_idx_form.append(vocab.word2index(word))
    if not invalid_ingredient:
        constraints_dict[ingredient] = ingredient_idx_form
    else:
        all_ingredients_list.remove(ingredient)
        num_invalid_ingredients += 1

In [9]:
all_ingredients_regex = get_ingredients_regex(all_ingredients_list)

In [10]:
test_dataloader = DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=pad_collate(vocab, train=False))

In [12]:
ingredients, recipes, ing_lens, _ = next(iter(test_dataloader))

In [15]:
ingredients_text = " ".join([vocab.index2word[i] for i in ingredients[0].tolist()])

In [16]:
input_ingredients = find_ingredients_in_text(ingredients_text, all_ingredients_regex)
input_ingredients

{'butter',
 'coconut',
 'confectioners sugar',
 'eggs',
 'ground nutmeg',
 'milk',
 'pie shell',
 'salt',
 'vanilla'}

In [17]:
input_ingrs_partial = ' '.join(input_ingredients).split()
input_ingrs_partial

['ground',
 'nutmeg',
 'eggs',
 'milk',
 'pie',
 'shell',
 'confectioners',
 'sugar',
 'vanilla',
 'butter',
 'coconut',
 'salt']

In [19]:
pos_constraints = [constraints_dict[ing] for ing in sorted(list(input_ingredients), key=len)]

In [20]:
neg_constraints = []
for constraint in constraints_dict.keys():
    if constraint in pos_constraints:
        continue
    valid_negative_constraint=True
    for word in constraint.split():
        if word in input_ingrs_partial:
            valid_negative_constraint = False
            break
    if valid_negative_constraint:
        neg_constraints.append(constraints_dict[constraint])

In [21]:
pos_constraints

[[23], [21], [30], [6], [31], [715], [772, 1254], [609, 442], [238, 16]]

In [22]:
neg_constraints

[[1089, 798, 1090, 1091],
 [1088, 1089, 798, 1098],
 [2479, 5331, 39, 340],
 [125, 126, 132, 133],
 [3315, 3316, 803, 340],
 [2840, 2486, 2905, 2378],
 [450, 125, 126, 133],
 [6470, 1854, 11874, 123],
 [1089, 798, 1098],
 [3315, 3316, 340],
 [1818, 1009, 307],
 [345, 365, 99],
 [2479, 39, 340],
 [623, 798, 1300],
 [181, 88, 89],
 [693, 1878, 307],
 [1125, 271, 376],
 [623, 1571, 822],
 [1089, 798, 1090],
 [1725, 39, 340],
 [798, 1090, 1091],
 [2512, 2513, 1983],
 [450, 132, 133],
 [791, 900, 1631],
 [181, 1571, 822],
 [2512, 2513, 1818],
 [440, 770, 20],
 [800, 784, 91],
 [1325, 2552, 2377],
 [21587, 21588, 1825],
 [271, 165, 91],
 [1089, 798, 4077],
 [447, 165, 91],
 [798, 3099, 1575],
 [971, 1586, 803],
 [2365, 2464, 2465],
 [8268, 3515, 340],
 [447, 1847, 340],
 [345, 365, 346],
 [813, 803, 716],
 [21587, 21588, 2331],
 [3323, 1586, 478],
 [1877, 3099, 1575],
 [354, 1584, 126],
 [3291, 1634, 29],
 [1061, 1082, 1629],
 [1288, 2409, 1323],
 [971, 1097, 1179],
 [1477, 236, 237],
 [2365

In [38]:
k=5
alpha=50 # likelihood
beta = 4 # num constraints satisfied
neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty = 10, 0.5, 0.5
lam = 5.0

encoder_attn.eval()
decoder_attn.eval()

with torch.no_grad():
    final_decoder_out_txt = eval_neuro_decoding_iter(ingredients, ing_lens, encoder_attn, decoder_attn,
                            vocab, pos_constraints, neg_constraints, max_recipe_len=MAX_RECIPE_LEN,
                            k=k, alpha=alpha, beta=beta, neg_constraint_penalty=neg_constraint_penalty,
                            likelihood_penalty=likelihood_penalty, low_irr_satisfaction_penalty=low_irr_satisfaction_penalty,
                            lam=lam)

In [40]:
recipes

[['<RECIPE_START>',
  'cream',
  'butter',
  'and',
  'sugar',
  'add',
  'egg',
  'yolks',
  ',',
  'one',
  'at',
  'a',
  'time',
  'and',
  'beat',
  'until',
  'mixture',
  'is',
  'very',
  'light',
  'and',
  'lemon',
  'colored',
  'add',
  'vanilla',
  'and',
  'milk',
  'in',
  'another',
  'bowl',
  'beat',
  'egg',
  'whites',
  'until',
  'foamy',
  'fold',
  'into',
  'creamed',
  'mixture',
  'fold',
  'in',
  '11/2',
  'cups',
  'of',
  'the',
  'coconut',
  'pour',
  'into',
  'baked',
  'pie',
  'shell',
  'and',
  'sprinkle',
  'with',
  'nutmeg',
  'top',
  'with',
  'remaining',
  'grated',
  'coconut',
  'bake',
  'in',
  'preheated',
  '350',
  'oven',
  'for',
  'about',
  '35',
  'minutes',
  'or',
  'until',
  'filling',
  'is',
  'just',
  'set',
  'serve',
  'warm',
  'or',
  'cold',
  'courtesy',
  'of',
  'dale',
  '&',
  'gail',
  'shipp',
  ',',
  'columbia',
  'md',
  '<RECIPE_END>']]

In [39]:
final_decoder_out_txt

['<RECIPE_START>',
 'eggs',
 ',',
 'milk',
 ',',
 'butter',
 ',',
 'vanilla',
 'and',
 'salt',
 'in',
 'a',
 'medium',
 '-',
 'size',
 'bowl',
 'gradually',
 'beat',
 'in',
 'sugar',
 'until',
 'stiff',
 'peaks',
 'form',
 'fold',
 'into',
 'coconut',
 'mixture',
 'pour',
 'into',
 'a',
 '9',
 'inch',
 'pie',
 'plate',
 'bake',
 'in',
 'a',
 'preheated',
 '350',
 'degree',
 'oven',
 'for',
 '30',
 '-',
 '35',
 'minutes',
 'or',
 'until',
 'set',
 'cool',
 'slightly',
 'before',
 'serving',
 '<RECIPE_END>']

In [35]:
final_decoder_out_txt

['<RECIPE_START>',
 'beat',
 'eggs',
 ',',
 'sugar',
 ',',
 'vanilla',
 ',',
 'milk',
 ',',
 'salt',
 'and',
 'vanilla',
 'in',
 'a',
 'medium',
 '-',
 'sized',
 'bowl',
 'and',
 'stir',
 'in',
 'the',
 'milk',
 'and',
 'vanilla',
 'pour',
 'into',
 'a',
 '9',
 '-',
 'inch',
 'pie',
 'plate',
 'bake',
 'in',
 'a',
 '350',
 'degree',
 'oven',
 'for',
 '30',
 '-',
 '40',
 'minutes',
 'or',
 'until',
 'set',
 'cool',
 'slightly',
 'before',
 'removing',
 'from',
 'pan',
 '<RECIPE_END>']

In [31]:
input_ingredients

{'butter',
 'coconut',
 'confectioners sugar',
 'eggs',
 'ground nutmeg',
 'milk',
 'pie shell',
 'salt',
 'vanilla'}

In [29]:
final_decoder_out_txt

['<RECIPE_START>',
 'cream',
 'butter',
 'and',
 'sugar',
 'until',
 'light',
 'and',
 'fluffy',
 'add',
 'eggs',
 'and',
 'vanilla',
 'beat',
 'until',
 '<RECIPE_END>']

---

In [11]:
E, H, V, L_i = 30, 20, 10, 5
# decoder = DecoderRNN(E, H, V)
decoder = AttnDecoderRNN(E, H, V, padding_val=999).to(DEVICE)
decoder_hidden = torch.rand([1, 1, H]).to(DEVICE)
decoder_cell = torch.rand([1, 1, H]).to(DEVICE)
encoder_houts = torch.rand([1, L_i, H]).to(DEVICE)
ingredients = torch.rand([1, L_i]).to(DEVICE)
pos_constraints = [[0, 1], [7], [6, 4, 5], [6], [2,4]]
neg_constraints = [[2], [4, 3], [9, 4, 3], [8]]
k, alpha, beta =3, 6, 2
neg_constraint_penalty, likelihood_penalty, low_irr_satisfaction_penalty = 0.5, 0.2, 0.1 # 2 and 3 probably needs to be much lower

In [34]:
eval_neuro_decoder_iter(decoder, decoder_hidden, decoder_cell, encoder_houts, ingredients, 10,
                        pos_constraints, neg_constraints, k, alpha, beta, neg_constraint_penalty,
                        likelihood_penalty, low_irr_satisfaction_penalty, lam=0.5)

([[2, 4, 7, 6, 4, 7, 4, 6, 4, 5, 3],
  [2, 4, 7, 6, 4, 7, 7, 7, 7, 4, 3],
  [2, 4, 7, 6, 4, 7, 7, 7, 7, 7, 3]],
 tensor([-1.7627, -1.7310, -1.7318], grad_fn=<DivBackward0>))

In [61]:
eval_neuro_decoder_iter(decoder, decoder_hidden, decoder_cell, encoder_houts, ingredients, 10,
                        pos_constraints, neg_constraints, k, alpha, beta, neg_constraint_penalty,
                        likelihood_penalty, low_irr_satisfaction_penalty, lam=1.0)

([[2, 4, 6, 4, 6, 4, 0, 0, 1, 7, 3],
  [2, 4, 6, 4, 6, 4, 0, 0, 7, 6, 3],
  [2, 4, 6, 4, 6, 4, 0, 0, 1, 6, 3]],
 tensor([-1.7299, -1.7107, -1.7056], grad_fn=<DivBackward0>))

In [35]:
eval_neuro_decoder_iter(decoder, decoder_hidden, decoder_cell, encoder_houts, ingredients, 10,
                        pos_constraints, neg_constraints, k, alpha, beta, neg_constraint_penalty,
                        likelihood_penalty, low_irr_satisfaction_penalty, lam=0.5)

([[2, 0, 1, 6, 7, 3], [2, 0, 1, 7, 6, 3], [2, 0, 1, 7, 3]],
 tensor([-1.7423, -1.7545, -1.6015], grad_fn=<DivBackward0>))

In [12]:
eval_neuro_decoder_iter(decoder, decoder_hidden, decoder_cell, encoder_houts, ingredients, 10,
                        pos_constraints, neg_constraints, k, alpha, beta, neg_constraint_penalty,
                        likelihood_penalty, low_irr_satisfaction_penalty, None)

([[2, 7, 3], [2, 6, 6, 6, 6, 6, 7, 6, 3], [2, 6, 6, 6, 3]],
 tensor([-1.2105, -1.7632, -1.4815], grad_fn=<DivBackward0>))

In [13]:
k = torch.rand([10])

In [15]:
k[range(3)]

tensor([0.0985, 0.3700, 0.4539])

In [17]:
k=torch.arange(3)[torch.tensor([0,2])][torch.tensor([0,1]).bool()]
k

tensor([2])

In [19]:
lstm = nn.LSTM(input_size=5, hidden_size=10, num_layers=3).to(DEVICE)

In [ ]:
hidden = torch.rand()

In [ ]:
lstm(torch.rand())